In [1]:
import os
import numpy as np
import pandas as pd

from datasets import load_dataset, load_from_disk

In [ ]:
use_downloads = False  # set this to False if the dataset is not downloaded to local disk

if use_downloads:
    downloads_data_path = "your/downloaded/original/tldr/data/path"
    train_dataset = load_from_disk(os.path.join(downloads_data_path, "train"))
    eval_dataset  = load_from_disk(os.path.join(downloads_data_path, "validation"))
else:  
    # download the openai/summarize_from_feedback dataset from huggingface
    # the dataset will be saved to ~/.cache/huggingface/datasets/
    train_dataset = load_dataset("openai/summarize_from_feedback", name="comparisons", split="train")
    eval_dataset  = load_dataset("openai/summarize_from_feedback", name="comparisons", split="validation")

print("train_dataset size:", len(train_dataset))
print("eval_dataset size:", len(eval_dataset))

train_dataset size: 92858
eval_dataset size: 86086


In [ ]:
# only use comparisons sampled from SFTs (but not ppo policies)
filter_func = lambda x: all(y["policy"].find("sup") >= 0 and y["policy"].find("ppo") == -1 and \
                            y["policy"].find("cnn") == -1 for y in x["summaries"])
train_dataset = train_dataset.filter(filter_func)  
eval_dataset  = eval_dataset.filter(filter_func)

print("train_dataset size:", len(train_dataset))
print("eval_dataset size:", len(eval_dataset))

train_dataset size: 38487
eval_dataset size: 16294


In [6]:
# annotation counts of each user
train_user, train_user_cnt = np.unique(np.asarray(train_dataset["worker"]), return_counts=True)
train_user = pd.DataFrame(np.column_stack((train_user, train_user_cnt)), columns=["worker", "train_count"])

eval_user, eval_user_cnt = np.unique(np.asarray(eval_dataset["worker"]), return_counts=True)
eval_user = pd.DataFrame(np.column_stack((eval_user, eval_user_cnt)), columns=["worker", "eval_count"])

In [7]:
# rank workers in descending order of the number of annotated comparisons
user = pd.merge(left=train_user, right=eval_user, how="outer", on="worker")
user.fillna(0, inplace=True)
user["train_count"] = user["train_count"].astype(int)
user["eval_count"] = user["eval_count"].astype(int)
user = user.sort_values(by=["train_count"], ascending=False)

user["train_count_acum"] = user["train_count"].cumsum()
user["eval_count_acum"] = user["eval_count"].cumsum()
user = user.reset_index(drop=True)

In [8]:
# top 10 workers with the highest annotation counts in the train split
user.iloc[:10]

,worker,train_count,eval_count,train_count_acum,eval_count_acum
0,KZL1qeRzHNYSfDAuOctL1iyVV8WC5N,5245,1175,5245,1175
1,ZzGCcAhvqF0HnKxNsUjtJFadcZdyZj,2920,643,8165,1818
2,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,2437,381,10602,2199
3,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,2366,604,12968,2803
4,zKV8BFGy60O0q7102ALF84S6Jo5i4q,2200,586,15168,3389
5,i8YiBZlrYmlkkChr5b9BUKvDO6lR1d,2126,360,17294,3749
6,M3icahkfAtC9CJrtKgQ7qvyZ5SD8wC,1824,321,19118,4070
7,HNzkrs9geGu1YMMfZ5Qvdt0ZaCthfB,1606,263,20724,4333
8,Jxv4hxfb9zTVa5nsMDFlnjSX5LZ8MK,1366,177,22090,4510
9,UhQipwcpQmiGJmScocXOGOKyCBaFUg,1209,411,23299,4921


In [9]:
# the top 10 / top 40 users for training
for n in [10, 40]:
    np.savetxt(f"./sup_users_top{n}.txt", user.iloc[:n]["worker"].to_numpy(), fmt="%s")
    print(f"top{n}", "train_count", user.iloc[n-1]["train_count_acum"], "eval_count", user.iloc[n-1]["eval_count_acum"])

top10 train_count 23299 eval_count 4921
top40 train_count 38065 eval_count 13060


In [10]:
# simulate preferences
# if preference = 0, user prefers longer summary
# if preference = 1, user prefers shorter summary
np.random.seed(3456)

user_preference = []
for i in range(user.shape[0]):
    # 70% probability to have preference = 0, and 30% to have preference = 1
    user_preference.append(np.random.choice(a=[0, 1], p=[0.7, 0.3]))

user["preference"] = user_preference

In [11]:
# preferences of seen users in the train/validation set
for n in [10, 40]: 
    table = pd.pivot_table(user.iloc[:n], index="preference", values=["train_count", "eval_count"], aggfunc=np.sum)
    table["eval_count%"] = 100*table["eval_count"]/table["eval_count"].sum()
    table["train_count%"] = 100*table["train_count"]/table["train_count"].sum()
    print(f"top {n} users")
    print(table)
    print("\n")

top 10 users
            eval_count  train_count  eval_count%  train_count%
preference                                                    
0                 3371        16607    68.502337     71.277737
1                 1550         6692    31.497663     28.722263


top 40 users
            eval_count  train_count  eval_count%  train_count%
preference                                                    
0                 8301        25821     63.56049     67.833968
1                 4759        12244     36.43951     32.166032




In [12]:
# preferences of unknown users in the validation set
for n in [10, 40]: 
    table = pd.pivot_table(user.iloc[n:], index="preference", values=["eval_count"], aggfunc=np.sum)
    table["eval_count%"] = 100*table["eval_count"]/table["eval_count"].sum()
    print(f"top {n} users")
    print(table)
    print("\n")

top 10 users
            eval_count  eval_count%
preference                         
0                 7237    63.633166
1                 4136    36.366834


top 40 users
            eval_count  eval_count%
preference                         
0                 2307    71.335807
1                  927    28.664193




In [13]:
# the saved sup_users_top10.txt and sup_users_preferences.txt files 
# will be used in the model training of "Generation with Conflicting Preferences" experiments
# using the TL;DR dataset with synthetic preferences
np.savetxt("./sup_users_preferences.txt", user[["worker", "preference"]].to_numpy(), fmt="%s")

In [14]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

In [15]:
from prlhf.utils import load_openai_comparisons

In [ ]:
# randomly sample 50 prompts from the validation set for evaluation
train_dataset, eval_dataset, n_users = load_openai_comparisons(
    user_file="./sup_users_top10.txt", 
    sep="||", 
    n_user_tokens=10,
    max_text_length=4800, 
    sanity_check=False,
    use_downloads=use_downloads,
    downloads_data_path=downloads_data_path if use_downloads else None,
    user_preference_file="./sup_users_preferences.txt",
)

print(len(train_dataset))
print(len(eval_dataset))
print(n_users)

46584
16294
10


In [18]:
unique_prompts = []  # the unique prompts in the validation set

for user_prompt in eval_dataset["prompt"]:
    prompt = user_prompt.split("<|endoftext|>||")[1].strip()
    if prompt in unique_prompts:
        continue
    unique_prompts.append(prompt)

print(len(unique_prompts))

3617


In [21]:
np.random.seed(123)
indices = np.random.choice(len(unique_prompts), 50, replace=False)

selected_prompts = np.asarray(unique_prompts)[indices]
print(len(selected_prompts))

# this saved npy file will provide the evaluation prompts for tldr generations
np.save("./tldr_selected_prompts.npy", selected_prompts)

50
